# Filtering stringtie predictions

Because fungal genes are so close together, it's fairly common for RNA-seq coverage to overlap between neighboring genes.
As a consequence, stringtie tends to create alternative transcripts that are actually just merges of two genes.
This is the problem we're trying to resolve here.

In [1]:
import re
import json
import pandas as pd
import numpy as np
from scipy.sparse import lil_array
from collections import defaultdict, Counter
from intervaltree import Interval, IntervalTree

from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree

from IPython.display import clear_output

In [2]:
LINEWIDTH=160

In [3]:
pd.set_option('display.width', LINEWIDTH)

In [4]:
regex1 = re.compile('transcript_id\s+"([^"]+)"')
regex2 = re.compile('ref_gene_id\s+"(?P<g>[^"]+)"')
regex3 = re.compile('gene_id\s+"([^"]+)"')

gtf = pd.read_csv(
    "stringtie_merged.gtf",
    sep="\t",
    comment="#",
    names="seqid source type start end score strand phase attributes".split(),
    low_memory=False
)

def get_ref_gene_id(f):
    m = regex2.search(f)

    if m is None:
        return pd.NA

    return m.groupdict().get("g")

gtf["start"] -= 1
gtf["transcript"] = gtf["attributes"].apply(lambda f: regex1.search(f).groups()[0])
gtf["gene_id"] = gtf["attributes"].apply(lambda f: regex3.search(f).groups()[0])
gtf["ref_gene_id"] = gtf["attributes"].apply(get_ref_gene_id)
gtf.drop(columns = "attributes", inplace=True)
gtf.sort_values(by=["seqid", "start", "end"], inplace=True)
gtf.head()

,seqid,source,type,start,end,score,strand,phase,transcript,gene_id,ref_gene_id
1,chr01,StringTie,exon,178,345,1000,-,.,MSTRG.1.1,MSTRG.1,<NA>
0,chr01,StringTie,transcript,178,1742,1000,-,.,MSTRG.1.1,MSTRG.1,<NA>
6,chr01,StringTie,exon,295,2294,1000,+,.,MSTRG.2.1,MSTRG.2,<NA>
5,chr01,StringTie,transcript,295,6460,1000,+,.,MSTRG.2.1,MSTRG.2,<NA>
11,chr01,StringTie,exon,312,882,1000,+,.,MSTRG.2.2,MSTRG.2,<NA>


First, we'll need a way to find overlapping genes.
This is relatively simple if we go by the gene names, but later on when we split genes this will be more useful.

In [5]:
gtf["transcript"].nunique()

24075

In [6]:
itrees = defaultdict(IntervalTree)
transcript_indices = dict()

for i, row in gtf[gtf["type"] == "transcript"].reset_index().iterrows():
    transcript = row["transcript"]
    transcript_indices[transcript] = i
    itrees[row["seqid"]].add(Interval(row["start"], row["end"], data=transcript))

Next we need a way of comparing two gene predictions.
We'll define the similarity as the jaccard index of the exon positions (ie. how similar is the overall intron-exon structure).
We'll also look as locus level merging based on a simple overlap coverage.

Ideally, we shouldn't have any cases where two loci that don't overlap, are covered by another locus that overlaps both.

In [7]:
transcripts = defaultdict(set)

for i, row in gtf[gtf["type"] == "transcript"].iterrows():
    transcript = row["transcript"]
    if row["strand"] == "-":
        strand = -1
        offset = -1
    else:
        strand = 1
        offset = 0

    transcripts[transcript].update(range((strand * row["start"]) + offset, (strand * row["end"]) + offset, strand))


exons = defaultdict(set)

for i, row in gtf[gtf["type"] == "exon"].iterrows():
    transcript = row["transcript"]
    if row["strand"] == "-":
        strand = -1
    else:
        strand = 1

    exons[transcript].update(range(strand * row["start"], strand * row["end"], strand))

In [8]:
def coverage(t1, t2):
    inter = len(t1.intersection(t2))
    return max([inter / len(t1), inter / len(t2)])

Now we'll calculate the pairwise coverages.

In [9]:
coverages = lil_array((len(transcript_indices), len(transcript_indices)), dtype=float)

for seqid, itree in itrees.items():
    for interval in sorted(itree):
        t1_name = interval.data
        t1_index = transcript_indices[t1_name]
        t1 = transcripts[t1_name]
        e1 = exons[t1_name]
        overlapping = itree.overlap(interval)
        for overlap in sorted(overlapping):
            t2_name = overlap.data
            t2_index = transcript_indices[t2_name]
            t2 = transcripts[t2_name]
            cov = coverage(t1, t2)
            coverages[t1_index, t2_index] = cov

From this we get a way to quickly compare two genes.

In [10]:
coverages[:5, :5].todense()

array([[1., 0., 0., 1., 0.],
       [0., 1., 1., 0., 1.],
       [0., 1., 1., 0., 1.],
       [1., 0., 0., 1., 0.],
       [0., 1., 1., 0., 1.]])

## Manual removal of loci that join distinct loci

Here we'll do a semi-manual curation of the transcripts.
Some cases are easy, while others are more in a grey area.
We'll look at the cases where it's a grey area and automatically deal with the easy ones.

- Joining transcripts are the problematic transcripts that we want to exclude.
- Joining exceptions are the transcripts that may look like problems, but we actually want to keep them in.

In [11]:
def is_bad(subdf, transcript_indices, coverages, good_prefix = "sscl"):
    gene_transcripts = subdf["transcript"].unique()
    gene_transcripts_indices = np.array([transcript_indices[t] for t in gene_transcripts])
    
    goodies = np.array([s.lower().startswith(good_prefix) for s in gene_transcripts])
    
    n = gene_transcripts.shape[0]
    col_index = np.repeat(gene_transcripts_indices, n)
    row_index = np.tile(gene_transcripts_indices, n)

    transcript_cov = np.reshape(
        coverages[row_index, col_index].todense(),
        (n, n)
    )
    
    transcript_cov = pd.DataFrame(transcript_cov, index=gene_transcripts, columns=gene_transcripts)

    if transcript_cov.size == 1:
        return transcript_cov, gene_transcripts, None, None

    if (transcript_cov.values < 1).any(axis=1).any():
        easy_cases = gene_transcripts[(transcript_cov.values > 0.99).all(axis=1) & (~goodies)]
        baddies = gene_transcripts[(transcript_cov.values > 0).all(axis=1) & (~goodies)]

        if np.size(easy_cases) == 0:
            easy_cases = None
        else:
            baddies = np.setdiff1d(baddies, easy_cases)
        
        if np.size(baddies) == 0:
            baddies = None
    else:
        baddies = None
        easy_cases = None

    
    return transcript_cov, gene_transcripts, baddies, easy_cases

In [12]:
def plot_exons(subdf, mapper=None, step=10, linewidth=80):
    subdf_exons = subdf[subdf["type"] == "exon"]
    start = subdf_exons["start"].min()
    end = subdf_exons["end"].max()

    seqs = dict()
    seqs["loc"] = "".join(f"{d: <10}" for d in range(start, end - step + 1, step))
    seqs["markers"] = "".join(["|" if i % 10 == 0 else " " for i in range(len(seqs["loc"]))])

    for tid, subdf2 in subdf.groupby("transcript"):
        subdf2_exons = subdf2[subdf2["type"] == "exon"]
        seq = np.array([" " for _ in range(end - start)])

        last_end = None
        for i, e in subdf2_exons.iterrows():
            seq[e["start"] - start: e["end"] - start] = e["strand"][0]
            if last_end is None:
                last_end = e["end"] - start
            else:
                seq[last_end: e["start"] - start] = "i"

            

        seq = "".join(seq)

        compressed_string = []
        for i in range(0, len(seq) - step + 1, step):
            p = seq[i:i + step].count("+")
            m = seq[i:i + step].count("-")
            g = seq[i:i + step].count(".")
            i = seq[i:i + step].count("i")

            if g >= p and g >= m and g >= i:
                compressed_string.append(".")
            elif i >= p and i >= m and i >= g:
                compressed_string.append("=")
            elif p >= g and p >= m and p >= i:
                compressed_string.append(">")
            elif m >= g and m >= p and m >= i:
                compressed_string.append("<")
            else:
                compressed_string.append("?")

        seqs[tid] = "".join(compressed_string)

    if mapper is not None:
        max_mapper_len = max([len(de) for de in mapper.values()]) + 2
    else:
        max_mapper_len = 0
        mapper = {id_: "" for id_ in seqs.keys()}
    
    max_id_len = max([len(id_) for id_ in seqs.keys()])
    
    for start in range(0, len(compressed_string) -  1, linewidth):
        end = start + linewidth
    
        for id_, seq in seqs.items():
            decision = mapper.get(id_, "")
            print(f"{decision:<{max_mapper_len}}{id_: >{max_id_len}}  ", seq[start:end])

        print()

In [13]:
def plot_transcripts(subdf, mapper=None, step=100, linewidth=80):
    subdf_transcripts = subdf[subdf["type"] == "transcript"]
    start = subdf_transcripts["start"].min()
    end = subdf_transcripts["end"].max()

    seqs = dict()
    seqs["loc"] = "".join(f"{d: <10}" for d in range(start, end - step + 1, step))
    seqs["markers"] = "".join(["|" if i % 10 == 0 else " " for i in range(len(seqs["loc"]))])

    for tid, subdf2 in subdf.groupby("transcript"):

        subdf2_transcript = subdf2[subdf2["type"] == "transcript"]
        seq = np.array([" " for _ in range(end - start)])

        for i, e in subdf2_transcript.iterrows():
            seq[e["start"] - start: e["end"] - start] = e["strand"][0]
            
        seq = "".join(seq)

        compressed_string = []
        for i in range(0, len(seq) - step + 1, step):
            p = seq[i:i + step].count("+")
            m = seq[i:i + step].count("-")
            g = seq[i:i + step].count(".")

            if g >= p and g >= m:
                compressed_string.append(".")
            elif p > g and p > m:
                compressed_string.append(">")
            elif m > g and m > p:
                compressed_string.append("<")
            else:
                raise ValueError("this shouldn't happen")

        seqs[tid] = "".join(compressed_string)

    if mapper is not None:
        max_mapper_len = max([len(de) for de in mapper.values()]) + 2
    else:
        max_mapper_len = 0
        mapper = {id_: "" for id_ in seqs.keys()}
    
    max_id_len = max([len(id_) for id_ in seqs.keys()])
    
    for start in range(0, len(compressed_string) -  1, linewidth):
        end = start + linewidth
    
        for id_, seq in seqs.items():
            decision = mapper.get(id_, "")
            print(f"{decision:<{max_mapper_len}}{id_: >{max_id_len}}  ", seq[start:end])

In [14]:
def get_bad_order(transcript_cov, bad_indices, good_prefix="sscl"):
    id_order = transcript_cov.sum(axis=1).sort_values(ascending=True).index
    id_order = id_order[~pd.isna(id_order)].tolist()
    if bad_indices is None:
        id_order = [
            c
            for c
            in transcript_cov
            if not c.lower().startswith(good_prefix)
        ]
    else:
        id_order = [
            c
            for c
            in transcript_cov
            if ((c in bad_indices) and not c.lower().startswith(good_prefix))
        ]

    return id_order

In [15]:
def get_response(worst, subdf=None, linewidth=80):
    response = None
    
    worst = list(reversed(worst))
    while response not in ("y", "n"):
        if len(worst) == 1:
            print(f"\nDelete [y] or keep [n] the worst option {worst[0]}.")
            print("You can also print the exon structure with [p].")
        else:
            print(f"\nDelete [y] or keep [n] the worst option {worst[0]}, select an index, keep all [r], or delete all [a].")
            print("You can also print the exon structure with [p].")
            for h, i in enumerate(worst):
                print(f"- {h}: {i}")

        response = input("\nresponse: ")
        response = response.strip().lower()

        try:
            response_int = int(response)
        except:
            response_int = None
        
        if response in ["n", "k"]:
            return worst[0], "keep"
        elif response in ["y", "d"]:
            return worst[0], "delete"
        elif response in ["a", "da"]:
            return worst, "delete_all"
        elif response in ["r", "ka"]:
            return worst, "keep_all"
        elif response in ["e", "p", "print", "exons"]:
            assert subdf is not None, "If printing exons you need to have provided the subdf"
            print("\nExons:")
            plot_exons(subdf, step=10, linewidth=linewidth)
            return get_response(worst, subdf=subdf, linewidth=linewidth)

        elif (response in worst) or ((response_int is not None) and (response_int < len(worst))):
            if response_int is not None:
                response = worst[response_int]

            response2 = "h"
            while response2 not in ("y", "n", "k", "d", "r"):
                response2 = input(f"\ndelete {response} [y], keep [n], or go back and reselect [r]?: ")

            if response2 in ("n", "k"):
                return response, "keep"
            elif response2 in ("y", "d"):
                return response, "delete"

In [16]:
joining_exceptions = set()
joining_transcripts = set()

In [17]:
with open("basic_overlap_filter.json", "r") as handle:
    d = json.load(handle)
    joining_exceptions = set(d["exceptions"])
    joining_transcripts = set(d["baddies"])

Now I want to check that the excluded and included entries are actually ok.
Mistakes happen.

In [18]:
gtf[gtf["transcript"].isin(joining_transcripts | joining_exceptions)].head()

,seqid,source,type,start,end,score,strand,phase,transcript,gene_id,ref_gene_id
6,chr01,StringTie,exon,295,2294,1000,+,.,MSTRG.2.1,MSTRG.2,<NA>
5,chr01,StringTie,transcript,295,6460,1000,+,.,MSTRG.2.1,MSTRG.2,<NA>
7,chr01,StringTie,exon,2374,2679,1000,+,.,MSTRG.2.1,MSTRG.2,<NA>
8,chr01,StringTie,exon,2738,5128,1000,+,.,MSTRG.2.1,MSTRG.2,<NA>
9,chr01,StringTie,exon,5186,6460,1000,+,.,MSTRG.2.1,MSTRG.2,<NA>


In [19]:
check_done = set()
check_responses = []

In [20]:
def get_check_response(mapper, subdf=None, linewidth=80):
    response = None
    # The unnecessary groupby is so that the order is the same as the plots.
    transcripts_tmp = set(subdf["transcript"])
    transcripts = list()

    for t in sorted(transcripts_tmp):
        transcripts.append([t, mapper[t], mapper[t]])

    while response not in ("y", "n"):
        print(f"\n")
        print("Please decide if your previous decisions were ok.")
        print("Enter [y]=yes/keep all, [n]=no/remove all anotations, or [e] to print exon structure.")
        print("Alternatively, you can enter a number to deal with an individual sequence.")

        max_mapper_len = max([len(j) for _, _, j in transcripts]) + 2
        max_id_len = max([len(i) for i, _, _ in transcripts])
        for h, (i, _, j) in enumerate(transcripts):
            print(f"- {j:<{max_mapper_len}}{i:>{max_id_len}}: {h}")

        response = input("\nresponse: ")
        response = response.strip().lower()

        try:
            response_int = int(response)
        except:
            response_int = None

        if response in ["n"]:
            return transcripts
        elif response in ["y"]:
            return transcripts
        elif response in ["e", "p", "print", "exons"]:
            assert subdf is not None, "If printing exons you need to have provided the subdf"
            print("\nExons:")
            plot_exons(subdf, step=20, mapper=mapper, linewidth=linewidth)

        elif (response_int is not None) and (response_int < len(transcripts)):
            response, prev, next_ = transcripts[response_int]

            print(f"\nEntry {response} was previously {prev}. What would you like to do with it?")

            response2 = "h"
            while response2 not in ("x", "d", "r"):
                response2 = input(f"\nEnter [x]=EXCEPTION, [d]=DELETED, [r]=RETAINED: ")
                response2 = response2.strip().lower()

            response2 = {"x": "EXCEPTION", "d": "DELETED", "r": "RETAINED"}[response2]
            transcripts[response_int][2] = response2
            mapper[transcripts[response_int][0]] = response2
    return

In [21]:
with open("basic_overlap_filter_check.json", "r") as handle:
    check_responses = json.load(handle)

In [22]:
responses = {
    tid
    for tid, _, decision
    in check_responses
    if decision == "DELETED"
}

Ok so i've curated a bunch of genes that are causing joining.
Now I need to assign new gene ids

## Another check for overlapping genes.

How many transcripts do we now have?

In [23]:
gtf.loc[~gtf["transcript"].isin(responses), "transcript"].nunique()

21893

In [24]:
gtf2 = gtf.loc[~gtf["transcript"].isin(responses),].reset_index(drop=True)
gtf2.head()

,seqid,source,type,start,end,score,strand,phase,transcript,gene_id,ref_gene_id
0,chr01,StringTie,exon,178,345,1000,-,.,MSTRG.1.1,MSTRG.1,<NA>
1,chr01,StringTie,transcript,178,1742,1000,-,.,MSTRG.1.1,MSTRG.1,<NA>
2,chr01,StringTie,exon,312,882,1000,+,.,MSTRG.2.2,MSTRG.2,<NA>
3,chr01,StringTie,transcript,312,4746,1000,+,.,MSTRG.2.2,MSTRG.2,<NA>
4,chr01,StringTie,transcript,415,1742,1000,-,.,sscle_01g000010,MSTRG.1,gene-sscle_01g000010


From here it's relatively easy to find overlapping sets.

In [25]:
itrees = defaultdict(IntervalTree)
transcript_indices = dict()
indices_transcript = list()

for i, row in gtf2[gtf2["type"] == "transcript"].reset_index().iterrows():
    transcript = row["transcript"]
    transcript_indices[transcript] = i
    indices_transcript.append(transcript)
    itrees[(row["seqid"], row["strand"])].add(Interval(row["start"], row["end"], data=transcript))

def reducer(x, y):
    if isinstance(x, str):
        x = {x}
    elif x is None:
        x = set()
        
    x.add(y)
    return x
    
for itree in itrees.values():
    itree.merge_overlaps(data_reducer=reducer, data_initializer=set())

In [26]:
new_genes = {}
i = 1

for itree in itrees.values():
    for interval in sorted(itree, key=lambda x: x.begin):
        gene_id = f"AS{i:0>5}"
        i += 1
        
        for tid in interval.data:
            new_genes[tid] = gene_id       

In [27]:
gtf2["new_gene_id"] = gtf2["transcript"].apply(new_genes.get)
gtf2.head()

,seqid,source,type,start,end,score,strand,phase,transcript,gene_id,ref_gene_id,new_gene_id
0,chr01,StringTie,exon,178,345,1000,-,.,MSTRG.1.1,MSTRG.1,<NA>,AS00001
1,chr01,StringTie,transcript,178,1742,1000,-,.,MSTRG.1.1,MSTRG.1,<NA>,AS00001
2,chr01,StringTie,exon,312,882,1000,+,.,MSTRG.2.2,MSTRG.2,<NA>,AS00775
3,chr01,StringTie,transcript,312,4746,1000,+,.,MSTRG.2.2,MSTRG.2,<NA>,AS00775
4,chr01,StringTie,transcript,415,1742,1000,-,.,sscle_01g000010,MSTRG.1,gene-sscle_01g000010,AS00001


Now i can find genes with multiple ref_gene_ids.

In [28]:
new_loci = gtf2[gtf2["ref_gene_id"].notnull()].groupby("new_gene_id")["transcript"].nunique()
new_loci[new_loci > 1]

new_gene_id
AS00292    2
AS00563    2
AS01432    2
AS01603    2
AS02952    2
AS04530    3
AS04709    3
AS05330    2
AS05956    2
AS06269    3
AS06672    5
AS07931    3
AS09271    3
AS09975    3
AS10013    3
AS10209    3
AS12354    2
AS12826    3
AS13467    2
Name: transcript, dtype: int64

In [29]:
gid = "AS00292"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()

print(x)

            loc   1461992   1462092   1462192   1462292   1462392   1462492   1462592   1462692   
        markers   |         |         |         |         |         |         |         |         
    MSTRG.460.3   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    MSTRG.460.4   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    MSTRG.460.5   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
sscle_01g004240   ..<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<..........................
sscle_01g004250   ......................................................<<<<<<<<<<
['MSTRG.460.3' 'MSTRG.460.4' 'MSTRG.460.5']


In [30]:
responses.update(x)

In [31]:
gid = "AS00563"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   2871825   2871925   2872025   2872125   2872225   2872325   2872425   2872525   
        markers   |         |         |         |         |         |         |         |         
    MSTRG.916.1   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    MSTRG.921.1   ................................................................................
sscle_01g008440   ................................................................................
sscle_01g008460   ................................................................................
            loc   2872625   2872725   2872825   2872925   2873025   2873125   2873225   2873325   
        markers   |         |         |         |         |         |         |         |         
    MSTRG.916.1   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    MSTRG.921.1   ...............<<<<<<<<<<<<<<<<<<<<.............................................
sscle_01g0

array(['MSTRG.916.1', 'MSTRG.921.1'], dtype=object)

In [32]:
responses.add("MSTRG.916.1")

In [33]:
gid = "AS01432"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   3519670   3519770   3519870   3519970   3520070   3520170   3520270   3520370   
        markers   |         |         |         |         |         |         |         |         
   MSTRG.1116.4   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>.............
sscle_01g010320   .....>>>>>>>>>>>>>>>>>>>>>>...............
sscle_01g010330   ..........................>>>>>>>>>>>>>>>>


array(['MSTRG.1116.4'], dtype=object)

In [34]:
m = gtf2.loc[gtf2["transcript"] == "sscle_01g010320", "end"].max()
gtf2.loc[gtf2["transcript"] == "MSTRG.1116.4", "end"] = gtf2.loc[gtf2["transcript"] == "MSTRG.1116.4", "end"].apply(lambda x: min([x, m]))

In [35]:
plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])

            loc   3519670   3519770   3519870   3519970   3520070   3520170   3520270   3520370   
        markers   |         |         |         |         |         |         |         |         
   MSTRG.1116.4   >>>>>>>>>>>>>>>>>>>>>>>>>>>...............
sscle_01g010320   .....>>>>>>>>>>>>>>>>>>>>>>...............
sscle_01g010330   ..........................>>>>>>>>>>>>>>>>


In [36]:
gid = "AS01603"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   443869    443969    444069    444169    444269    444369    444469    444569    
        markers   |         |         |         |         |         |         |         |         
   MSTRG.1409.1   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
   MSTRG.1409.2   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
sscle_02g012870   ...>>>>>>>....................................................................
sscle_02g012880   ........................>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.......


array(['MSTRG.1409.1', 'MSTRG.1409.2'], dtype=object)

In [37]:
responses.update(x)

In [38]:
gid = "AS02952"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   171434    171534    171634    171734    171834    171934    172034    172134    
        markers   |         |         |         |         |         |         |         |         
   MSTRG.2500.1   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.........................
   MSTRG.2500.4   ................................>>>>>>>>>>>>>>>>>>>>>>>>>>
sscle_03g022720   ........>>>>>>>>>>>>>>>>>>>>>>>...........................
sscle_03g022730   ..............................>>>>>>>>>>>>>>>>>>>>>>>>>>>>


array(['MSTRG.2500.1', 'MSTRG.2500.4'], dtype=object)

In [39]:
m = gtf2.loc[gtf2["transcript"] == "sscle_03g022720", "end"].max()
gtf2.loc[gtf2["transcript"] == "MSTRG.2500.1", "end"] = gtf2.loc[gtf2["transcript"] == "MSTRG.2500.1", "end"].apply(lambda x: min([x, m]))

In [40]:
gtf2[(gtf2["new_gene_id"] == gid) & (gtf2["type"] == "transcript")]

,seqid,source,type,start,end,score,strand,phase,transcript,gene_id,ref_gene_id,new_gene_id
17638,chr03,StringTie,transcript,171434,174482,1000,+,.,MSTRG.2500.1,MSTRG.2500,<NA>,AS02952
17644,chr03,StringTie,transcript,172249,174482,1000,+,.,sscle_03g022720,MSTRG.2500,gene-sscle_03g022720,AS02952
17650,chr03,StringTie,transcript,174482,177315,1000,+,.,sscle_03g022730,MSTRG.2500,gene-sscle_03g022730,AS02952
17652,chr03,StringTie,transcript,174704,177315,1000,+,.,MSTRG.2500.4,MSTRG.2500,<NA>,AS02952


In [41]:
gid = "AS04530"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   1702477   1702577   1702677   1702777   1702877   1702977   1703077   1703177   
        markers   |         |         |         |         |         |         |         |         
   MSTRG.4104.1   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>...............................
   MSTRG.4104.3   ............>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.........
   MSTRG.4104.4   .............>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
   MSTRG.4104.6   .....................>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.........
sscle_04g036970   >>>>>>>>>>>>>..................................................................
sscle_04g036980   ................>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>...............................
sscle_04g036990   .....................................................................>>>>>>>>>>


array(['MSTRG.4104.1', 'MSTRG.4104.3', 'MSTRG.4104.4', 'MSTRG.4104.6'],
      dtype=object)

In [42]:
responses.update(["MSTRG.4104.1", "MSTRG.4104.4"])

In [43]:
gid = "AS05956"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   378853    378953    379053    379153    379253    379353    379453    379553    
        markers   |         |         |         |         |         |         |         |         
   MSTRG.4595.3   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
   MSTRG.4595.4   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
sscle_05g041550   .>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.................................
sscle_05g041560   ............................................>>>>>>>>>>>>>>>>...


array(['MSTRG.4595.3', 'MSTRG.4595.4'], dtype=object)

In [44]:
responses.update(x)

In [45]:
gid = "AS06269"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   2123980   2124080   2124180   2124280   2124380   2124480   2124580   2124680   
        markers   |         |         |         |         |         |         |         |         
   MSTRG.5144.1   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>...........
   MSTRG.5144.2   .>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.......................
   MSTRG.5144.5   ...........................................................>>>>>>>>>>>>>>>>>>>>>
sscle_05g046700   ..>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>..........................................
sscle_05g046710   ........................................>>>>>>>>>>>>>>..........................
sscle_05g046730   ....................................................................>>>>>>>>>>>>
            loc   2124780   2124880   2124980   2125080   2125180   2125280   2125380   2125480   
        markers   |         |         |         |         |         |         |         |         
   MSTRG.5

array(['MSTRG.5144.1', 'MSTRG.5144.2', 'MSTRG.5144.5'], dtype=object)

In [46]:
responses.update(['MSTRG.5144.1', 'MSTRG.5144.2'])

In [47]:
gid = "AS06672"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   1450195   1450295   1450395   1450495   1450595   1450695   1450795   1450895   
        markers   |         |         |         |         |         |         |         |         
   MSTRG.5814.1   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
   MSTRG.5814.4   ........................................................<<<<<<<<<<<<<<<<<<<<<<<<
   MSTRG.5814.7   ................................................................................
sscle_06g052560   <<<<<<<<<<<<<<<<<<<<<<<<<<<.....................................................
sscle_06g052570   ..........................<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<.......................
sscle_06g052580   ...............................................................<<<<<<<<.........
sscle_06g052590   ........................................................................<<<<<<<<
sscle_06g052610   ................................................................................
          

array(['MSTRG.5814.1', 'MSTRG.5814.4', 'MSTRG.5814.7'], dtype=object)

In [48]:
responses.update(['MSTRG.5814.1', 'MSTRG.5814.4'])

In [49]:
gid = "AS07931"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   776485    776585    776685    776785    776885    776985    777085    777185    
        markers   |         |         |         |         |         |         |         |         
   MSTRG.6394.1   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.................................................
   MSTRG.6394.2   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>...................
   MSTRG.6394.3   .>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.................................................
   MSTRG.6394.5   ..............................>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
   MSTRG.6394.7   ................................................................>>>>>>>>>>>>>>>>
   MSTRG.6394.8   .................................................................>>>>>>>>>>>>>>>
sscle_07g057500   ...>>>>>>>>>>>>>>>>>>>>>>>>>>>>.................................................
sscle_07g057510   ...................................>>>>>>>>>>>>>>>>>>>>>>>>>>...................
sscle_07g0

array(['MSTRG.6394.1', 'MSTRG.6394.2', 'MSTRG.6394.3', 'MSTRG.6394.5',
       'MSTRG.6394.7', 'MSTRG.6394.8'], dtype=object)

In [50]:
responses.update(['MSTRG.6394.2', 'MSTRG.6394.5'])

In [51]:
gid = "AS09271"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   923942    924042    924142    924242    924342    924442    924542    924642    
        markers   |         |         |         |         |         |         |         |         
   MSTRG.7965.1   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
   MSTRG.7965.2   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
   MSTRG.7965.3   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
   MSTRG.7965.4   ........>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
   MSTRG.7965.6   .......................>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>......
   MSTRG.7965.7   ................................................................................
   MSTRG.7965.8   ................................................................................
   MSTRG.7965.9   ................................................................................
sscle_09g0

array(['MSTRG.7965.1', 'MSTRG.7965.2', 'MSTRG.7965.3', 'MSTRG.7965.4',
       'MSTRG.7965.6', 'MSTRG.7965.7', 'MSTRG.7965.8', 'MSTRG.7965.9'],
      dtype=object)

In [52]:
responses.update(['MSTRG.7965.4', 'MSTRG.7965.7', 'MSTRG.7965.8'])

In [53]:
gid = "AS09975"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   432000    432100    432200    432300    432400    432500    432600    432700    
        markers   |         |         |         |         |         |         |         |         
   MSTRG.8454.1   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
   MSTRG.8454.2   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
   MSTRG.8454.3   ..<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
   MSTRG.8454.5   ...................................<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
   MSTRG.8454.6   .........................................................<<<<<<<<<<<<<<<<<<<<<<<
   MSTRG.8454.8   ................................................................................
   MSTRG.8454.9   ................................................................................
sscle_10g076010   ..<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<..............................................
sscle_10g0

array(['MSTRG.8454.1', 'MSTRG.8454.2', 'MSTRG.8454.3', 'MSTRG.8454.5',
       'MSTRG.8454.6', 'MSTRG.8454.8', 'MSTRG.8454.9'], dtype=object)

In [54]:
responses.update(['MSTRG.8454.1', 'MSTRG.8454.2', 'MSTRG.8454.3', 'MSTRG.8454.5'])

In [55]:
gid = "AS10013"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   667135    667235    667335    667435    667535    667635    667735    667835    
        markers   |         |         |         |         |         |         |         |         
   MSTRG.8526.1   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<.........
   MSTRG.8526.3   ..................<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
sscle_10g076660   <<<<<<<<<<<<<<<<<<<................................
sscle_10g076670   ..........................<<<<<<<<<<<<<<...........
sscle_10g076680   .........................................<<<<<<<<<<


array(['MSTRG.8526.1', 'MSTRG.8526.3'], dtype=object)

In [56]:
responses.update(x)

In [57]:
gid = "AS10209"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   1785334   1785434   1785534   1785634   1785734   1785834   1785934   1786034   
        markers   |         |         |         |         |         |         |         |         
   MSTRG.8858.1   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<...........
   MSTRG.8858.3   ............................................<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
sscle_10g079910   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<...................................
sscle_10g079920   .............................................<<<<<<<<<<<<<<<....................
sscle_10g079930   ......................................................................<<<<<<<<<.
            loc   1786134   1786234   1786334   1786434   1786534   1786634   1786734   1786834   
        markers   |         |         |         |         |         |         |         |         
   MSTRG.8858.1   .......
   MSTRG.8858.3   <<<<<<<
sscle_10g079910   .......
sscle_10g079920   .......
sscle

array(['MSTRG.8858.1', 'MSTRG.8858.3'], dtype=object)

In [58]:
responses.update(x)

In [59]:
gid = "AS12354"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   1134981   1135081   1135181   1135281   1135381   1135481   1135581   1135681   
        markers   |         |         |         |         |         |         |         |         
  MSTRG.10565.3   <<<<<<<<<<<<<<<<<<
  MSTRG.10565.4   ...<<<<<<<<<<<<<<<
sscle_13g095060   ....<<<<<<<<......
sscle_13g095070   ............<<<<<<


array(['MSTRG.10565.3', 'MSTRG.10565.4'], dtype=object)

In [60]:
responses.update(x)

In [61]:
gid = "AS12826"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   16876     16976     17076     17176     17276     17376     17476     17576     
        markers   |         |         |         |         |         |         |         |         
  MSTRG.10767.3   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
  MSTRG.10769.1   .................................<<<<<<<........................................
sscle_14g097040   ............................<<<<................................................
sscle_14g097060   ..............................................................<<<<<<<<<<<<<<<<<<
sscle_14g097070   ................................................................................
            loc   17676     17776     17876     17976     18076     18176     18276     18376     
        markers   |         |         |         |         |         |         |         |         
  MSTRG.10767.3   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
  MSTRG.10

array(['MSTRG.10767.3', 'MSTRG.10769.1'], dtype=object)

In [62]:
responses.add('MSTRG.10767.3')

In [63]:
gid = "AS13467"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   1754363   1754463   1754563   1754663   1754763   1754863   1754963   1755063   
        markers   |         |         |         |         |         |         |         |         
  MSTRG.11301.1   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.................
  MSTRG.11301.2   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.....................
  MSTRG.11301.3   .>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>................
  MSTRG.11301.5   ......................................>>>>>>>>>>>>>>>>>>
sscle_14g102030   .................>>>>>>>>>..............................
sscle_14g102040   ........................................>>>>>>>>>>>>>>>>


array(['MSTRG.11301.1', 'MSTRG.11301.2', 'MSTRG.11301.3', 'MSTRG.11301.5'],
      dtype=object)

In [64]:
responses.add("MSTRG.11301.3")

In [65]:
gid = "AS04709"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   2613504   2613604   2613704   2613804   2613904   2614004   2614104   2614204   
        markers   |         |         |         |         |         |         |         |         
   MSTRG.4385.1   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>...........
   MSTRG.4385.2   .>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>...........
   MSTRG.4385.3   .>>>>>>>>>>>>>>>>>>>...............................................
   MSTRG.4385.5   ...................>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
sscle_04g039620   ........>>>>>>>>>>>>...............................................
sscle_04g039630   .........................>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>...........
sscle_04g039640   .........................................................>>>>>>>>>>


array(['MSTRG.4385.1', 'MSTRG.4385.2', 'MSTRG.4385.3', 'MSTRG.4385.5'],
      dtype=object)

In [66]:
responses.update(['MSTRG.4385.1', 'MSTRG.4385.2', 'MSTRG.4385.5'])

In [67]:
gid = "AS05330"

plot_transcripts(gtf2[gtf2["new_gene_id"] == gid])
x = gtf2.loc[(gtf2["new_gene_id"] == gid) & pd.isnull(gtf2["ref_gene_id"]), "transcript"].unique()
x

            loc   94381     94481     94581     94681     94781     94881     94981     95081     
        markers   |         |         |         |         |         |         |         |         
   MSTRG.4491.1   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
   MSTRG.4491.2   ................................................................................
sscle_05g040600   ..........................<<<<<<<<..............................................
sscle_05g040640   ................................................................................
            loc   95181     95281     95381     95481     95581     95681     95781     95881     
        markers   |         |         |         |         |         |         |         |         
   MSTRG.4491.1   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
   MSTRG.4491.2   ................................<<<<<<<<<<<<<
sscle_05g040600   .............................................
sscle_05g040640 

array(['MSTRG.4491.1', 'MSTRG.4491.2'], dtype=object)

In [68]:
responses.add("MSTRG.4491.1")

Now i'm looking for components with low coverage.

In [69]:
gtf3 = gtf2.loc[~gtf2["transcript"].isin(responses),].reset_index(drop=True)
gtf3.head()

itrees = defaultdict(IntervalTree)
transcript_indices = dict()
indices_transcript = list()

for i, row in gtf3[gtf3["type"] == "transcript"].reset_index().iterrows():
    transcript = row["transcript"]
    transcript_indices[transcript] = i
    indices_transcript.append(transcript)
    itrees[(row["seqid"], row["strand"])].add(Interval(row["start"], row["end"], data=transcript))

def reducer(x, y):
    if isinstance(x, str):
        x = {x}
    elif x is None:
        x = set()
        
    x.add(y)
    return x
    
for itree in itrees.values():
    itree.merge_overlaps(data_reducer=reducer, data_initializer=set())
    
new_genes = {}
i = 1

for itree in itrees.values():
    for interval in sorted(itree, key=lambda x: x.begin):
        gene_id = f"AS{i:0>5}"
        i += 1
        
        for tid in interval.data:
            new_genes[tid] = gene_id

gtf3["new_gene_id2"] = gtf3["transcript"].apply(new_genes.get)
gtf3.head()

,seqid,source,type,start,end,score,strand,phase,transcript,gene_id,ref_gene_id,new_gene_id,new_gene_id2
0,chr01,StringTie,exon,178,345,1000,-,.,MSTRG.1.1,MSTRG.1,<NA>,AS00001,AS00001
1,chr01,StringTie,transcript,178,1742,1000,-,.,MSTRG.1.1,MSTRG.1,<NA>,AS00001,AS00001
2,chr01,StringTie,exon,312,882,1000,+,.,MSTRG.2.2,MSTRG.2,<NA>,AS00775,AS00777
3,chr01,StringTie,transcript,312,4746,1000,+,.,MSTRG.2.2,MSTRG.2,<NA>,AS00775,AS00777
4,chr01,StringTie,transcript,415,1742,1000,-,.,sscle_01g000010,MSTRG.1,gene-sscle_01g000010,AS00001,AS00001


In [70]:
new_loci = gtf3[gtf3["ref_gene_id"].notnull()].groupby("new_gene_id2")["transcript"].nunique()
new_loci[new_loci > 1]

Series([], Name: transcript, dtype: int64)

In [71]:
check2_done = set()
check2_responses = []

In [72]:
def intersect(a, b):
    astart, aend = a
    bstart, bend = b
    
    assert astart < aend, "a"
    assert bstart < bend, "b"
    ostart = max([astart, bstart])
    oend = min([aend, bend])
    
    if ostart > oend:
        return 0
    else:
        return oend - ostart

def union(a, b):
    astart, aend = a
    bstart, bend = b
    
    assert astart < aend, "a"
    assert bstart < bend, "b"
    #ostart = min([astart, bstart])
    #oend = max([aend, bend])
    al = aend - astart
    bl = bend - bstart
    return (al + bl) - intersect(a, b)

In [73]:
def gene_coverage(subdf):
    subdf = subdf[subdf["type"] == "transcript"].reset_index()
    mat = np.zeros((subdf.shape[0], subdf.shape[0]))
    
    for i, rowi in subdf.iterrows():
        for j, rowj in subdf.iterrows():
            inter = intersect((rowi["start"], rowi["end"]), (rowj["start"], rowj["end"]))
            uni = union((rowi["start"], rowi["end"]), (rowj["start"], rowj["end"]))
            min_ = min([abs(rowi["start"] - rowi["end"]), abs(rowj["start"] - rowj["end"])])
            mat[i, j] = inter / min_

    mat = pd.DataFrame(mat, columns=subdf["transcript"], index=subdf["transcript"])
    return mat

In [75]:
with open("basic_overlap_filter_check2.json", "r") as handle:
    check2_responses = json.load(handle)

responses = {
    tid
    for tid, _, decision
    in check2_responses
    if decision == "DELETED"
}

In [76]:
gtf4 = gtf3.loc[~gtf3["transcript"].isin(responses),].reset_index(drop=True)
gtf4.head()

itrees = defaultdict(IntervalTree)
transcript_indices = dict()
indices_transcript = list()

for i, row in gtf4[gtf4["type"] == "transcript"].reset_index().iterrows():
    transcript = row["transcript"]
    transcript_indices[transcript] = i
    indices_transcript.append(transcript)
    itrees[(row["seqid"], row["strand"])].add(Interval(row["start"], row["end"], data=transcript))

def reducer(x, y):
    if isinstance(x, str):
        x = {x}
    elif x is None:
        x = set()
        
    x.add(y)
    return x
    
for itree in itrees.values():
    itree.merge_overlaps(data_reducer=reducer, data_initializer=set())
    
new_genes = {}
i = 1

for itree in itrees.values():
    for interval in sorted(itree, key=lambda x: x.begin):
        gene_id = f"AS{i:0>5}"
        i += 1
        
        for tid in interval.data:
            new_genes[tid] = gene_id

gtf4["new_gene_id3"] = gtf4["transcript"].apply(new_genes.get)
gtf4.head()

,seqid,source,type,start,end,score,strand,phase,transcript,gene_id,ref_gene_id,new_gene_id,new_gene_id2,new_gene_id3
0,chr01,StringTie,exon,178,345,1000,-,.,MSTRG.1.1,MSTRG.1,<NA>,AS00001,AS00001,AS00001
1,chr01,StringTie,transcript,178,1742,1000,-,.,MSTRG.1.1,MSTRG.1,<NA>,AS00001,AS00001,AS00001
2,chr01,StringTie,exon,312,882,1000,+,.,MSTRG.2.2,MSTRG.2,<NA>,AS00775,AS00777,AS00777
3,chr01,StringTie,transcript,312,4746,1000,+,.,MSTRG.2.2,MSTRG.2,<NA>,AS00775,AS00777,AS00777
4,chr01,StringTie,transcript,415,1742,1000,-,.,sscle_01g000010,MSTRG.1,gene-sscle_01g000010,AS00001,AS00001,AS00001


In [77]:
def jaccard(t1, t2):
    return (len(t1.intersection(t2)) / len(t1.union(t2)))

def jaccard_cov(df1, df2):
    one = set()
    two = set()
    
    for i, row in df1[df1["type"] == "exon"].iterrows():
        one.update(range(row["start"], row["end"]))
        
    for i, row in df2[df2["type"] == "exon"].iterrows():
        two.update(range(row["start"], row["end"]))
    
    return jaccard(one, two)


def gene_similarities(subdf):
    subdf_transcripts = subdf[subdf["type"] == "transcript"].drop_duplicates().reset_index()
    mat = pd.DataFrame(
        np.zeros((subdf_transcripts.shape[0], subdf_transcripts.shape[0])),
        columns=subdf_transcripts["transcript"],
        index=subdf_transcripts["transcript"]
    )    

    for i, subdfi in subdf.groupby("transcript"):
        for j, subdfj in subdf.groupby("transcript"):
            c = jaccard_cov(subdfi, subdfj)

            mat.loc[i, j] = c

    return mat

In [99]:
def get_check_response_jaccard(mapper, subdf=None, linewidth=80):
    response = None

    transcripts_li = sorted(subdf["transcript"].unique())
    transcripts = {t: "RETAINED" for t in transcripts_li}

    while response not in ("y", "n"):
        print(f"\n")
        print("Please decide if your previous decisions were ok.")
        print("Enter [y]=yes/keep all, [n]=no/remove all anotations, [f] to flag for followup, [d] to show the dataframe, or [e] to print exon structure.")
        print("Alternatively, you can enter a number to deal with an individual sequence.")

        max_mapper_len = max([len(v) for k, v in transcripts.items()]) + 2
        max_id_len = max([len(k) for k, v in transcripts.items()])
        for h, (k, v) in enumerate(transcripts.items()):
            print(f"- {v:<{max_mapper_len}}{k:>{max_id_len}}: {h}")

        response = input("\nresponse: ")
        response = response.strip().lower()

        try:
            response_int = int(response)
        except:
            response_int = None

        if response in ["n"]:
            return {k: "DELETED" for k, v in transcripts.items()}
        elif response in ["y"]:
            return transcripts
        elif response in ["f"]:
            return {k: "FOLLOWUP" for k, v in transcripts.items()}
        elif response in ["d"]:
            assert subdf is not None, "no dataframe"
            print("Dataframe")
            print(subdf)
        elif response in ["e", "p", "print", "exons"]:
            assert subdf is not None, "If printing exons you need to have provided the subdf"
            print("\nExons:")
            plot_exons(subdf, step=20, mapper=mapper, linewidth=linewidth)

        elif (response_int is not None) and (response_int < len(transcripts)):
            response = transcripts_li[response_int]
            prev = transcripts[response]

            print(f"\nEntry {response} was previously {prev}. What would you like to do with it?")

            response2 = "h"
            while response2 not in ("x", "d", "r"):
                response2 = input(f"\nEnter [x]=EXCEPTION, [d]=DELETED, [r]=RETAINED, [f]=FOLLOWUP: ")
                response2 = response2.strip().lower()

            response2 = {"x": "EXCEPTION", "d": "DELETED", "r": "RETAINED", "f": "FOLLOWUP"}[response2]
            transcripts[response] = response2
            mapper[response] = response2
    return

In [104]:
done = set()

mapper = {t: "RETAINED" for t in gtf4["transcript"].unique()}
jaccard_checks = dict()

In [114]:
non_overlapping = set()

In [119]:
for gene_id, subdf in gtf4.groupby("new_gene_id3"):
    if subdf["transcript"].nunique() == 1:
        continue
    sim = gene_similarities(subdf)

    if np.any(sim.values == 0):
        non_overlapping.add(gene_id)

    if gene_id in done:
        continue

    if np.any(sim.values < 0.4):
        print("\n", gene_id)
        print(sim)
        plot_exons(subdf, mapper=mapper, linewidth=160)
        print()

        responses = get_check_response_jaccard(mapper, subdf=subdf, linewidth=160)
        jaccard_checks.update(responses)
        clear_output()
    else:
        jaccard_checks.update({tid: mapper[tid] for tid in subdf["transcript"].unique()})

    done.add(gene_id)

with open("basic_overlap_filter_jaccard.json", "w") as handle:
    json.dump(jaccard_checks, handle)

In [120]:
with open("basic_overlap_filter_jaccard.json", "r") as handle:
    jaccard_checks = json.load(handle)

I happen to know that no transcripts additional genes needed to be excluded by the jaccard filter, but we did have some transcripts that existed entirely within the introns of other genes.
These are those genes:

In [123]:
non_overlapping

{'AS00936',
 'AS01045',
 'AS02146',
 'AS04979',
 'AS07322',
 'AS07332',
 'AS07649',
 'AS11048',
 'AS11312',
 'AS13480',
 'AS14405',
 'AS14408'}

We'll mark them.

In [128]:
gtf_final = gtf4.copy()
gtf_final.drop(columns=["new_gene_id", "new_gene_id2", "ref_gene_id", "gene_id"], inplace=True)

In [137]:
gtf_final_order = (
    gtf_final
    [gtf_final["type"] == "transcript"]
    .groupby("new_gene_id3")
    [["seqid", "start", "end"]]
    .apply(lambda x: pd.Series({"seqid": x.iloc[0, 0], "start": x["start"].min(), "end": x["end"].max()}))
    .reset_index()
    .sort_values(["seqid", "start", "end"])
    .reset_index()
)

In [141]:
gtf_final_gene_mapper = {
    j: f"AS{i:0>5}" for i, j in enumerate(gtf_final_order["new_gene_id3"])
}

In [142]:
gtf_final["gene_id"] = gtf_final["new_gene_id3"].apply(gtf_final_gene_mapper.get)

In [147]:
gtf_final.sort_values(["seqid", "start", "end", "gene_id"], inplace=True)

In [169]:
gtf_final["orig_transcript_id"] = gtf_final["transcript"].apply(lambda x: x if x.startswith("sscle") else pd.NA)
gene_id_mapper = gtf_final[gtf_final["orig_transcript_id"].notnull()][["gene_id", "orig_transcript_id"]].drop_duplicates().set_index("gene_id")["orig_transcript_id"].to_dict()

gtf_final["orig_gene_id"] = gtf_final["gene_id"].apply(lambda x: f"gene-{gene_id_mapper.get(x, pd.NA)}")
gtf_final["has_intronic_nonoverlapping"] = gtf_final["new_gene_id3"].apply(lambda x: x in non_overlapping)

In [173]:
def mapper(row):
    out = []
    out.append(f'transcript_id "{row["transcript"]}"')
    out.append(f'gene_id "{row["gene_id"]}"')

    if pd.notnull(row["orig_gene_id"]):
        out.append(f'orig_gene_id "{row["orig_gene_id"]}"')
    
    if pd.notnull(row["orig_transcript_id"]):
        out.append(f'orig_transcript_id "{row["orig_transcript_id"]}"')
    
    if row["has_intronic_nonoverlapping"]:
        out.append(f'has_intronic_nonoverlapping "true"')

    return "; ".join(out)
        
gtf_final["attributes"] = gtf_final.apply(mapper, axis = 1)
gtf_final

,seqid,source,type,start,end,score,strand,phase,transcript,new_gene_id3,gene_id,orig_transcript_id,orig_gene_id,has_intronic_nonoverlapping,attributes
0,chr01,StringTie,exon,178,345,1000,-,.,MSTRG.1.1,AS00001,AS00000,<NA>,gene-sscle_01g000010,False,"transcript_id ""MSTRG.1.1""; gene_id ""AS00000""; ..."
1,chr01,StringTie,transcript,178,1742,1000,-,.,MSTRG.1.1,AS00001,AS00000,<NA>,gene-sscle_01g000010,False,"transcript_id ""MSTRG.1.1""; gene_id ""AS00000""; ..."
2,chr01,StringTie,exon,312,882,1000,+,.,MSTRG.2.2,AS00777,AS00001,<NA>,gene-sscle_01g000020,False,"transcript_id ""MSTRG.2.2""; gene_id ""AS00001""; ..."
3,chr01,StringTie,transcript,312,4746,1000,+,.,MSTRG.2.2,AS00777,AS00001,<NA>,gene-sscle_01g000020,False,"transcript_id ""MSTRG.2.2""; gene_id ""AS00001""; ..."
4,chr01,StringTie,transcript,415,1742,1000,-,.,sscle_01g000010,AS00001,AS00000,sscle_01g000010,gene-sscle_01g000010,False,"transcript_id ""sscle_01g000010""; gene_id ""AS00..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86003,chr16,StringTie,transcript,1416950,1417793,1000,+,.,sscle_16g111300,AS14748,AS14750,sscle_16g111300,gene-sscle_16g111300,False,"transcript_id ""sscle_16g111300""; gene_id ""AS14..."
86004,chr16,StringTie,exon,1417136,1417319,1000,+,.,sscle_16g111300,AS14748,AS14750,sscle_16g111300,gene-sscle_16g111300,False,"transcript_id ""sscle_16g111300""; gene_id ""AS14..."
86005,chr16,StringTie,exon,1417369,1417539,1000,+,.,sscle_16g111300,AS14748,AS14750,sscle_16g111300,gene-sscle_16g111300,False,"transcript_id ""sscle_16g111300""; gene_id ""AS14..."
86006,chr16,StringTie,exon,1417605,1417672,1000,+,.,sscle_16g111300,AS14748,AS14750,sscle_16g111300,gene-sscle_16g111300,False,"transcript_id ""sscle_16g111300""; gene_id ""AS14..."


In [ ]:
gtf_final.to_csv("stringtie_gtf_processed.gtf", sep="\t", header=False, index=False, na_rep=".", )

In [178]:
(
    gtf_final
    ["seqid source type start end score strand phase attributes".split()]
    .to_csv("stringtie_gtf_processed.gtf", sep="\t", header=False, index=False, na_rep=".", quoting=3)
)